# Correlation with Meter Pricing/Zones

This notebook investigates the relationship between violation rates and meter pricing/zones. We will merge datasets, visualize rates by zone, state a hypothesis, and summarize findings.

## 1. Import Required Libraries
We will use pandas, numpy, matplotlib, seaborn for analysis and visualization.

## 2. Relevant Datasets for Meter Pricing/Zones Analysis

- **MTA Bus Automated Camera Enforcement Violations** ([CSV](https://data.ny.gov/api/views/kh8p-hcbm/rows.csv?accessType=DOWNLOAD))
- **NYC DOT Parking Meter Zones** ([CSV](https://data.cityofnewyork.us/api/views/ei4h-7p3j/rows.csv?accessType=DOWNLOAD))
- **NYC DOT Parking Meter Rates** ([CSV](https://data.cityofnewyork.us/api/views/ncbg-4pzn/rows.csv?accessType=DOWNLOAD))

We will attempt to join violations with meter zones/rates to analyze violation rates by pricing/zone.

## 3. Analysis Plan: Exploring Correlation with Meter Pricing/Zones

1. **Explore Columns and Data Types**
   - Print all columns, data types, and preview rows for each dataset.
   - Identify possible join keys (e.g., location, zone, meter ID, street).
2. **Join Datasets for Analysis**
   - Attempt to join violations with meter zones/rates.
   - If not possible, suggest geocoding or fuzzy matching.
3. **Visualize Violation Rates by Zone/Pricing**
   - Bar plots, heatmaps, and maps of violation rates by zone and pricing.
   - Compare rates across different pricing levels and locations.
4. **Aggregate and Rank Zones**
   - Group by zone or pricing to find top areas for violations.
   - Visualize with bar charts or tables.
5. **Temporal and Categorical Patterns**
   - Analyze if certain zones have more violations at specific times or for certain violation types.
6. **Formulate Hypotheses**
   - Example: "Higher meter rates are associated with lower violation rates."
   - Example: "Certain zones have more violations during peak hours."
7. **Summarize Findings and Recommendations**
   - List key patterns, possible causes, and actionable insights.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style='whitegrid')

In [32]:
# Load the MTA Bus Automated Camera Enforcement Violations dataset
violations_url = "https://data.ny.gov/api/views/kh8p-hcbm/rows.csv?accessType=DOWNLOAD"
violations_df = pd.read_csv(violations_url, nrows=10000)

print('Columns in violations_df:')
print(violations_df.columns.tolist())
display(violations_df.head())

Columns in violations_df:
['Violation ID', 'Vehicle ID', 'First Occurrence', 'Last Occurrence', 'Violation Status', 'Violation Type', 'Bus Route ID', 'Violation Latitude', 'Violation Longitude', 'Stop ID', 'Stop Name', 'Bus Stop Latitude', 'Bus Stop Longitude', 'Violation Georeference', 'Bus Stop Georeference']


,Violation ID,Vehicle ID,First Occurrence,Last Occurrence,Violation Status,Violation Type,Bus Route ID,Violation Latitude,Violation Longitude,Stop ID,Stop Name,Bus Stop Latitude,Bus Stop Longitude,Violation Georeference,Bus Stop Georeference
0,489749182,c5ae1411153b52556a1e648cc80d718aa519a4bdd189ab...,08/20/2025 11:12:08 PM,08/21/2025 12:24:08 AM,TECHNICAL ISSUE/OTHER,MOBILE BUS STOP,BX36,40.840509,-73.881189,102498,EAST TREMONT AV/VYSE AV,40.841076,-73.882483,POINT (-73.881189 40.840509),POINT (-73.882483 40.841076)
1,489744714,df9044acf85cf55488aea4cd3ce1d0e17ef050551726b6...,08/20/2025 11:48:59 PM,08/20/2025 11:54:47 PM,EXEMPT - BUS/PARATRANSIT,MOBILE BUS STOP,BX28,40.874017,-73.890646,100080,PAUL AV/BEDFORD PARK BLVD,40.874629,-73.891539,POINT (-73.890646 40.874017),POINT (-73.891539 40.874629)
2,489743631,eb5a337966ba65f66ab1db8e169d2446a4fb429b0efc63...,08/20/2025 10:33:13 PM,08/20/2025 11:56:02 PM,TECHNICAL ISSUE/OTHER,MOBILE DOUBLE PARKED,Q53+,40.721971,-73.867136,550473,WOODHAVEN BLVD/PENELOPE AV,40.722487,-73.867736,POINT (-73.867136 40.721971),POINT (-73.867736 40.722487)
3,489741945,3f877f70d9b253515a945be807c9c62d5814949f810310...,08/20/2025 10:50:45 PM,08/20/2025 11:32:43 PM,EXEMPT - OTHER,MOBILE BUS STOP,Q44+,40.762529,-73.831728,501140,UNION ST/35 AV,40.765422,-73.827944,POINT (-73.831728 40.762529),POINT (-73.827944 40.765422)
4,489741940,7feac037b62d591ffb1214e356157f3dd197fc22fee5bb...,08/20/2025 10:52:57 AM,08/20/2025 11:16:57 AM,EXEMPT - EMERGENCY VEHICLE,MOBILE BUS STOP,M101,40.815113,-73.955040,401458,AMSTERDAM AV/W 131 ST,40.816009,-73.954424,POINT (-73.95504 40.815113),POINT (-73.954424 40.816009)


In [33]:
# Load the NYC DOT Parking Meter Zones dataset
import urllib.error

try:
	meter_zones_df = pd.read_csv(meter_zones_url)
	print('Columns in meter_zones_df:')
	print(meter_zones_df.columns.tolist())
	display(meter_zones_df.head())
except urllib.error.HTTPError as e:
	print(f"HTTP Error: {e}. The dataset URL may be incorrect or unavailable.")
except Exception as e:
	print(f"An error occurred: {e}")

HTTP Error: HTTP Error 404: Not Found. The dataset URL may be incorrect or unavailable.


In [34]:
# Load the NYC DOT Parking Meter Rates dataset
import urllib.error

try:
	meter_rates_df = pd.read_csv(meter_rates_url)
	print('Columns in meter_rates_df:')
	print(meter_rates_df.columns.tolist())
	display(meter_rates_df.head())
except urllib.error.HTTPError as e:
	print(f"HTTP Error: {e}. The dataset URL may be incorrect or unavailable.")
except Exception as e:
	print(f"An error occurred: {e}")

HTTP Error: HTTP Error 404: Not Found. The dataset URL may be incorrect or unavailable.


In [35]:
# Diagnostic: Check non-null and unique counts for likely join keys in each dataset
likely_keys = ['zone', 'meter', 'meter_id', 'street', 'location', 'id']
for name in ['violations_df', 'meter_zones_df', 'meter_rates_df']:
    if name in locals():
        df = locals()[name]
        print(f'\n--- Diagnostics for {name} ---')
        for key in likely_keys:
            matches = [col for col in df.columns if key in col.lower()]
            for col in matches:
                non_null = df[col].notnull().sum()
                unique = df[col].nunique()
                print(f"{col}: non-null={non_null}, unique={unique}")
    else:
        print(f"{name} is not loaded. Please check the dataset URL or previous cell output.")


--- Diagnostics for violations_df ---
Violation ID: non-null=10000, unique=10000
Vehicle ID: non-null=9992, unique=5981
Bus Route ID: non-null=10000, unique=40
Stop ID: non-null=10000, unique=1450
meter_zones_df is not loaded. Please check the dataset URL or previous cell output.
meter_rates_df is not loaded. Please check the dataset URL or previous cell output.


## 4. Exploratory Data Analysis (EDA) and Join Key Exploration

We will:
- Print info, describe, and missing values for each dataset
- Explore possible join keys by comparing columns and sample values

In [36]:
# EDA: Info, describe, and missing values for all datasets
datasets = []
if 'violations_df' in locals():
    datasets.append(('violations_df', violations_df))
if 'meter_zones_df' in locals():
    datasets.append(('meter_zones_df', meter_zones_df))
if 'meter_rates_df' in locals():
    datasets.append(('meter_rates_df', meter_rates_df))

for name, df in datasets:
    print(f'\n--- {name} ---')
    print('Shape:', df.shape)
    print('Columns:', df.columns.tolist())
    print(df.info())
    display(df.describe(include='all'))
    print('Missing values per column:')
    print(df.isnull().sum())


--- violations_df ---
Shape: (10000, 15)
Columns: ['Violation ID', 'Vehicle ID', 'First Occurrence', 'Last Occurrence', 'Violation Status', 'Violation Type', 'Bus Route ID', 'Violation Latitude', 'Violation Longitude', 'Stop ID', 'Stop Name', 'Bus Stop Latitude', 'Bus Stop Longitude', 'Violation Georeference', 'Bus Stop Georeference']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Violation ID            10000 non-null  int64  
 1   Vehicle ID              9992 non-null   object 
 2   First Occurrence        10000 non-null  object 
 3   Last Occurrence         10000 non-null  object 
 4   Violation Status        10000 non-null  object 
 5   Violation Type          10000 non-null  object 
 6   Bus Route ID            10000 non-null  object 
 7   Violation Latitude      10000 non-null  float64
 8   Violation Longitude

,Violation ID,Vehicle ID,First Occurrence,Last Occurrence,Violation Status,Violation Type,Bus Route ID,Violation Latitude,Violation Longitude,Stop ID,Stop Name,Bus Stop Latitude,Bus Stop Longitude,Violation Georeference,Bus Stop Georeference
count,1.000000e+04,9992,10000,10000,10000,10000,10000,10000.000000,10000.000000,10000.000000,10000,10000.000000,10000.000000,10000,10000
unique,NaN,5981,9451,9848,6,3,40,NaN,NaN,NaN,1163,NaN,NaN,9982,1450
top,NaN,6d7caef8bf15e2bf8d46aa1e6dd8c0da1ae2fbc5b31ace...,08/18/2025 08:33:31 PM,08/15/2025 09:02:00 AM,EXEMPT - EMERGENCY VEHICLE,MOBILE BUS STOP,M101,NaN,NaN,NaN,AMSTERDAM AV/W 175 ST,NaN,NaN,POINT (-73.952192 40.648147),POINT (-73.934076 40.84381)
freq,NaN,53,3,3,3528,4782,1687,NaN,NaN,NaN,163,NaN,NaN,3,163
mean,4.892335e+08,NaN,NaN,NaN,NaN,NaN,NaN,40.773722,-73.927835,359390.635800,NaN,40.774009,-73.927762,NaN,NaN
std,2.967441e+05,NaN,NaN,NaN,NaN,NaN,NaN,0.072538,0.051631,170879.782726,NaN,0.072950,0.051984,NaN,NaN
min,4.887390e+08,NaN,NaN,NaN,NaN,NaN,NaN,40.537638,-74.164882,100017.000000,NaN,40.534540,-74.184125,NaN,NaN
25%,4.889139e+08,NaN,NaN,NaN,NaN,NaN,NaN,40.727770,-73.958900,302765.750000,NaN,40.730389,-73.958772,NaN,NaN
50%,4.893142e+08,NaN,NaN,NaN,NaN,NaN,NaN,40.783585,-73.937789,402112.000000,NaN,40.785480,-73.937851,NaN,NaN
75%,4.894931e+08,NaN,NaN,NaN,NaN,NaN,NaN,40.834591,-73.901369,405069.000000,NaN,40.835522,-73.901203,NaN,NaN


Missing values per column:
Violation ID              0
Vehicle ID                8
First Occurrence          0
Last Occurrence           0
Violation Status          0
Violation Type            0
Bus Route ID              0
Violation Latitude        0
Violation Longitude       0
Stop ID                   0
Stop Name                 0
Bus Stop Latitude         0
Bus Stop Longitude        0
Violation Georeference    0
Bus Stop Georeference     0
dtype: int64


In [37]:
# Ensure required variables are defined
required_vars = ['meter_zones_url', 'meter_rates_url', 'violations_cols', 'violations_df']
missing_vars = [v for v in required_vars if v not in locals()]
if missing_vars:
    print(f"Missing required variables: {missing_vars}. Please run the previous cells that define them.")
else:
    # Load meter_zones_df if not already loaded
    if 'meter_zones_df' not in locals():
        try:
            meter_zones_df = pd.read_csv(meter_zones_url)
        except Exception as e:
            print(f"An error occurred while loading meter_zones_df: {e}")

    # Load meter_rates_df if not already loaded
    if 'meter_rates_df' not in locals():
        try:
            meter_rates_df = pd.read_csv(meter_rates_url)
        except Exception as e:
            print(f"An error occurred while loading meter_rates_df: {e}")

    # Only proceed if both meter_zones_df and meter_rates_df are defined
    if 'meter_zones_df' in locals() and 'meter_rates_df' in locals():
        meter_zones_cols = set(meter_zones_df.columns.str.lower())
        meter_rates_cols = set(meter_rates_df.columns.str.lower())

        print('Columns in violations_df:', violations_cols)
        print('Columns in meter_zones_df:', meter_zones_cols)
        print('Columns in meter_rates_df:', meter_rates_cols)

        # Find common columns
        common_zones = violations_cols & meter_zones_cols
        common_rates = violations_cols & meter_rates_cols
        print('\nPossible join keys between violations and meter_zones:', common_zones)
        print('Possible join keys between violations and meter_rates:', common_rates)

        # Print sample values for likely join columns
        likely_keys = ['zone', 'meter', 'street', 'location', 'id']
        for key in likely_keys:
            for df, name in zip([violations_df, meter_zones_df, meter_rates_df], ['violations_df', 'meter_zones_df', 'meter_rates_df']):
                matches = [col for col in df.columns if key in col.lower()]
                for col in matches:
                    print(f"Sample values for {col} in {name}:")
                    print(df[col].dropna().unique()[:5])
    else:
        print("meter_zones_df and/or meter_rates_df are not defined. Please check previous cells for errors in loading these datasets.")

An error occurred while loading meter_zones_df: HTTP Error 404: Not Found
An error occurred while loading meter_rates_df: HTTP Error 404: Not Found
meter_zones_df and/or meter_rates_df are not defined. Please check previous cells for errors in loading these datasets.
An error occurred while loading meter_rates_df: HTTP Error 404: Not Found
meter_zones_df and/or meter_rates_df are not defined. Please check previous cells for errors in loading these datasets.


## 5. Data Cleaning and Standardization

We will standardize column names, clean key columns, and handle missing values to prepare for joining and analysis.

In [ ]:
# Data cleaning: standardize column names, trim whitespace, handle missing values
def clean_df(df):
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    for col in df.select_dtypes(include='object').columns:
        df[col] = df[col].astype(str).str.strip()
    return df

violations_df = clean_df(violations_df)

# Ensure meter_zones_df and meter_rates_df are loaded before cleaning
if 'meter_zones_df' not in locals():
    try:
        meter_zones_df = pd.read_csv(meter_zones_url)
    except Exception as e:
        print(f"An error occurred while loading meter_zones_df: {e}")

if 'meter_rates_df' not in locals():
    try:
        meter_rates_df = pd.read_csv(meter_rates_url)
    except Exception as e:
        print(f"An error occurred while loading meter_rates_df: {e}")

if 'meter_zones_df' in locals():
    meter_zones_df = clean_df(meter_zones_df)
if 'meter_rates_df' in locals():
    meter_rates_df = clean_df(meter_rates_df)

# Optionally drop duplicates and handle missing values
dfs = [violations_df]
if 'meter_zones_df' in locals():
    dfs.append(meter_zones_df)
if 'meter_rates_df' in locals():
    dfs.append(meter_rates_df)
for df in dfs:
    df.drop_duplicates(inplace=True)
    # You can also fill or drop missing values as needed
    # df.fillna('Unknown', inplace=True)

print('Data cleaning complete. Sample columns:')
print('violations_df:', violations_df.columns.tolist())
if 'meter_zones_df' in locals():
    print('meter_zones_df:', meter_zones_df.columns.tolist())
else:
    print('meter_zones_df not loaded.')
if 'meter_rates_df' in locals():
    print('meter_rates_df:', meter_rates_df.columns.tolist())
else:
    print('meter_rates_df not loaded.')

An error occurred while loading meter_zones_df: HTTP Error 404: Not Found
An error occurred while loading meter_rates_df: HTTP Error 404: Not Found


NameError: name 'meter_zones_df' is not defined

## 6. Join Datasets and Diagnose Joins

We will attempt to join violations with meter zones and rates on plausible keys, and print diagnostics for join success.

In [ ]:
# Attempt joins on plausible keys and print diagnostics
join_keys = [
    ('zone', 'zone'),
    ('meter_id', 'meter_id'),
    ('street', 'street'),
    ('location', 'location')
]

joined = None
for vkey, zkey in join_keys:
    if vkey in violations_df.columns and zkey in meter_zones_df.columns:
        joined = violations_df.merge(meter_zones_df, left_on=vkey, right_on=zkey, how='left', suffixes=('', '_zone'))
        print(f"Join on {vkey} and {zkey}: {joined.shape[0]} rows, {joined[vkey].notnull().sum()} matched.")
        break
if joined is None:
    print("No direct join possible on standard keys. Consider fuzzy matching or geocoding.")
else:
    # Try to join with meter rates if possible
    for rkey in ['zone', 'meter_id', 'location']:
        if rkey in joined.columns and rkey in meter_rates_df.columns:
            joined = joined.merge(meter_rates_df, on=rkey, how='left', suffixes=('', '_rate'))
            print(f"Further joined with meter_rates_df on {rkey}. New shape: {joined.shape}")
            break
    display(joined.head())

In [ ]:
# Fallback: If join fails, aggregate and visualize by street or location columns
if 'joined' not in locals() or joined is None:
    print('No successful join. Aggregating by street/location columns in violations_df.')
    fallback_keys = [col for col in violations_df.columns if any(k in col.lower() for k in ['street', 'location'])]
    for key in fallback_keys:
        counts = violations_df[key].value_counts().head(20)
        print(f"Top 20 by {key}:")
        print(counts)
        plt.figure(figsize=(10,6))
        sns.barplot(x=counts.values, y=counts.index, orient='h')
        plt.title(f'Top 20 {key} by Violation Count')
        plt.xlabel('Violation Count')
        plt.ylabel(key)
        plt.tight_layout()
        plt.show()
else:
    print('Join succeeded; see previous analysis.')

## 7. Exploratory Visualizations and Aggregation

We will visualize violation counts by zone, rate, and time, and aggregate/rank top zones and rates.

In [ ]:
# Bar plot: Violation counts by zone (if available)
if 'zone' in violations_df.columns:
    zone_counts = violations_df['zone'].value_counts().head(20)
    plt.figure(figsize=(10,6))
    sns.barplot(x=zone_counts.values, y=zone_counts.index, orient='h')
    plt.title('Top 20 Zones by Violation Count')
    plt.xlabel('Violation Count')
    plt.ylabel('Zone')
    plt.tight_layout()
    plt.show()
else:
    print('Zone column not found in violations_df.')

# Bar plot: Violation counts by meter rate (if available after join)
if 'rate' in (joined.columns if 'joined' in locals() else []):
    rate_counts = joined['rate'].value_counts().head(20)
    plt.figure(figsize=(10,6))
    sns.barplot(x=rate_counts.values, y=rate_counts.index, orient='h')
    plt.title('Top 20 Meter Rates by Violation Count')
    plt.xlabel('Violation Count')
    plt.ylabel('Meter Rate')
    plt.tight_layout()
    plt.show()
else:
    print('Rate column not found in joined DataFrame.')

## 8. Correlation, Regression, and Hypothesis Summary

We will check for correlation between meter rates and violation counts, and summarize findings with a logical hypothesis.

In [ ]:
# Correlation and regression between meter rate and violation count (if available)
if 'rate' in (joined.columns if 'joined' in locals() else []) and 'zone' in joined.columns:
    # Aggregate by rate
    agg = joined.groupby('rate').size().reset_index(name='violation_count')
    print(agg.head())
    # Correlation
    try:
        agg['rate_numeric'] = pd.to_numeric(agg['rate'], errors='coerce')
        corr = agg[['rate_numeric', 'violation_count']].corr().iloc[0,1]
        print(f'Correlation between meter rate and violation count: {corr:.2f}')
    except Exception as e:
        print(f'Could not compute correlation: {e}')
    # Simple regression
    try:
        import statsmodels.api as sm
        X = agg['rate_numeric'].dropna()
        y = agg.loc[X.index, 'violation_count']
        X = sm.add_constant(X)
        model = sm.OLS(y, X).fit()
        print(model.summary())
    except Exception as e:
        print(f'Could not run regression: {e}')
else:
    print('Rate or zone column not found in joined DataFrame. Skipping correlation/regression.')

# Hypothesis and summary (edit after running analysis)
hypothesis = '''
- Hypothesis: Higher meter rates are associated with [lower/higher/no change in] violation rates.
- Key findings: [Summarize patterns, correlations, and top zones/rates.]
- Actionable insights: [Suggest enforcement, pricing, or policy recommendations.]
'''
print(hypothesis)

## 9. Summary, Hypothesis, and Actionable Insights

- **Hypothesis:** There is a relationship between meter pricing/zones and violation rates. Higher rates may deter violations, or certain zones may have unique patterns.
- **Key Findings:**
    - Summarize whether joins were successful and which keys worked best.
    - Note any strong patterns by zone, rate, or street/location.
    - Highlight any correlations or lack thereof.
- **Actionable Insights:**
    - Recommend targeted enforcement or pricing adjustments for high-violation zones.
    - Suggest further data collection or geocoding if joins were not possible.
    - Propose policy or design changes based on observed patterns.

*Edit this cell after running the analysis to reflect your actual findings.*